# Sección

El propósito del código es leer datos de un archivo de Excel, convertirlo a un formato adecuado y luego guardar los datos en una base de datos SQLite y en un archivo JSON.

In [ ]:
# 1. Importar las bibliotecas necesarias:
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine

In [ ]:
# 2. Crear una conexión a la base de datos SQLite `presupuesto.db`:
engine = create_engine('sqlite:///presupuesto.db', echo=False)
# 3. Leer el archivo de Excel `file.xlsx` y almacenar cada hoja en un DataFrame de pandas en un diccionario llamado `df`:
df = pd.read_excel('file.xlsx', sheet_name=None, na_filter=False)

In [ ]:
# 4. Definir la función `cast_for_gsheets` para ajustar el tipo de datos en los DataFrames y guardarlos en la base de datos:

def cast_for_gsheets(df):
    for key in df.keys():
      # Guardar cada DataFrame en una tabla de la base de datos
      df[key].to_sql(key, con=engine)

      # Rellenar los valores NaN con strings vacías
      df[key] = df[key].fillna('')
      for column, dt in zip(df[key].columns, df[key].dtypes):
          if dt.type not in [
              np.int64,
              np.float_,
              np.bool_,
          ]:
              # Convertir los valores de las columnas a strings
              df[key].loc[:, column] = df[key][column].astype(str)
    return df


In [ ]:

# 5. Llamar a la función `cast_for_gsheets` y guardar el resultado en un nuevo diccionario `newdf`:
newdf = cast_for_gsheets(df)

# 6. Convertir cada DataFrame en `newdf` en un diccionario de registros y almacenarlos en un diccionario `data_dict`:
data_dict = {
    key: newdf[key].to_dict(orient='records') 
    for key in newdf.keys()
}

# 7. Escribir el contenido de `data_dict` en un archivo JSON `data_dict.json`:
with open('data_dict.json', 'w') as fp:
    json.dump(
        data_dict,
        fp,
        indent=4,
        sort_keys=True,
        ensure_ascii=False
    )

# 8. Consultar y mostrar los nombres de las tablas en la base de datos SQLite:
with engine.begin() as conn:
   conn.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()